In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
ls data

men_shoes.csv


In [5]:
df = pd.read_csv('data/men_shoes.csv')
mens_prices_amountmin_filter = 895.0
df_f = df[df.prices_amountmin <  mens_prices_amountmin_filter]
df_f.shape
df = df_f

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [7]:
df['brand_cat'] = (df['brand'].map(lambda x: str(x).lower())).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [8]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
# feature enginering
def parse_features(x):
  output_dict = {}
  if(str(x) == 'nan'): return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [10]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [11]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [12]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)  

In [13]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_lens color family', 'feat_foot arch', 'feat_watch case shape',
       'feat_construction', 'feat_inseam', 'feat_stock no.',
       'feat_primary shelf id', 'feat_labor', 'feat_country/ organization',
       'feat_safety feature'],
      dtype='object', length=526)

In [0]:
keys_stats = {}
for key in keys:
  keys_stats[key] = (df[df[get_name_feat(key)].isnull() == False].shape[0] / df.shape[0])*100

In [15]:
{k:v for k,v in keys_stats.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [17]:
#df[ df.brand == df.feat_brand ][ ['brand', 'feat_brand']].head()
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 1002)

In [18]:
feats = ['']

model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [19]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_shoe category',
 'feat_clothing category',
 'feat_certifications and listings',
 'feat_recommended location',
 'feat_location - country',
 'feat_fabrication',
 'feat_location - city/state',
 'feat_multi pack indicator',
 'feat_catalog',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_best sellers rank_cat',
 'feat_nike id_cat',
 'feat_shoe category_cat',
 'feat_battery voltage_cat',
 'feat_dial material_cat',
 'feat_overall material_cat',
 'feat_sole_cat',
 'feat_seller inventory number_cat',
 'feat_instrument_cat',
 'feat_front style_cat',
 'feat_type_cat',
 'feat_insulated_cat',
 'feat_lens width_cat',
 'feat_bezel_cat',
 'feat_light transmission (vlt)_cat',
 'feat_width_cat',
 'feat_ring size_cat',
 'feat_model number_cat',
 'feat_theme_cat',
 'feat_fabric material_cat',
 'feat_temple size:_cat',
 'feat_spikes type_cat',
 'feat_countr

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
feats = ['brand_cat', 'feat_brand_cat', 'feat_metal type_cat', 'feat_shape_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
#feats += feats_cat
#feats = list(set(feats))
result = run_model(feats, model)

In [0]:
X = df[feats].values
y = df['prices_amountmin'].values
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=1)
model.fit(X, y)
print(result)
perm = PermutationImportance(model, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

(-57.33110470067281, 4.2167542650926295)


Weight,Feature
0.2632 ± 0.0081,brand_cat
0.1027 ± 0.0071,feat_material_cat
0.0228 ± 0.0024,feat_gender_cat
0.0183 ± 0.0010,feat_brand_cat
0.0140 ± 0.0008,feat_shape_cat
0.0093 ± 0.0012,feat_metal type_cat
0.0028 ± 0.0003,feat_style_cat
0.0002 ± 0.0001,feat_sport_cat


In [0]:
df['brand'].value_counts(normalize=True)

nike              0.097210
puma              0.033315
ralph lauren      0.028775
vans              0.021116
new balance       0.020295
                    ...   
snugpak           0.000055
trugel            0.000055
rance             0.000055
grabbers          0.000055
tenneco monroe    0.000055
Name: brand, Length: 1732, dtype: float64

In [0]:
df[ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
measure_units = {'lbs' : 454, 'g' : 1, 'pounds': 454, 'ounces': 28.35, 'kg': 1000, 'lb': 454}
def process_weight(x):
  arr = str(x).lower().split();
  if(len(arr) > 1):
    weight = arr[0]
    units = arr[1]
    return str(round(float(weight)*measure_units[units]))
  return str(x)  

df['weight_processed'] = df['weight'].map(process_weight)
df['weight_processed_cat'] = df['weight_processed'].factorize()[0]

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
feats = ['brand_cat', 'feat_brand_cat', 'feat_metal type_cat', 'feat_shape_cat', 'weight_processed_cat', 'feat_gender_cat', 'feat_material_cat']
result = run_model(feats, model)

In [53]:
X = df[feats].values
y = df['prices_amountmin'].values
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=1)
model.fit(X, y)
print(result)
perm = PermutationImportance(model, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

(-57.2478451906678, 4.1625024268091995)


Weight,Feature
0.2681 ± 0.0125,brand_cat
0.0997 ± 0.0099,feat_material_cat
0.0231 ± 0.0021,feat_gender_cat
0.0179 ± 0.0008,feat_brand_cat
0.0144 ± 0.0009,feat_shape_cat
0.0083 ± 0.0010,feat_metal type_cat
0.0032 ± 0.0005,weight_processed_cat
